1. Сгенерируйте массив A из N=1млн случайных целых чисел на отрезке от 0 до 1000. Пусть B[i] = A[i] + 100. Посчитайте среднее значение массива B.

In [1]:
import random

N = 1000000
A = [random.randint(0, 1000) for i in range(N)]
B = [num + 100 for num in A]

average_B = sum(B) / len(B)
print(average_B)

600.133871


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец key, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце key указаны первые 5 английских букв.

In [2]:
import pandas as pd
import numpy as np

df = pd.DataFrame(np.random.rand(2000000, 4), columns=['col1', 'col2', 'col3', 'col4'])
df['key'] = np.random.choice(list('ABCDEFGHIJKLMNOPQRSTUVWXYZ'), size=2000000)

print(df.head())

       col1      col2      col3      col4 key
0  0.341593  0.735396  0.552466  0.955983   U
1  0.418118  0.797559  0.131570  0.825313   F
2  0.909958  0.257973  0.388731  0.377044   H
3  0.191254  0.852633  0.708756  0.441200   F
4  0.598709  0.836978  0.626206  0.185735   E


## Лабораторная работа 3

1. В файлах recipes_sample.csv и reviews_sample.csv (ЛР 2) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде pd.DataFrame с названиями recipes и reviews. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.
Реализуйте несколько вариантов функции подсчета среднего значения столбца rating из таблицы reviews для отзывов, оставленных в 2010 году.

A. С использованием метода DataFrame.iterrows исходной таблицы;

Б. С использованием метода DataFrame.iterrows таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода Series.mean.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций

In [3]:
import pandas as pd
import time
recipes = pd.read_csv('recipes_sample.csv', index_col=0)
reviews = pd.read_csv('reviews_sample.csv', index_col=0)

recipes['id'] = recipes['id'].astype(int)
recipes['minutes'] = recipes['minutes'].astype(int)

reviews['id'] = reviews['id'].astype(int)
reviews['user_id'] = reviews['user_id'].astype(int)
reviews['rating'] = reviews['rating'].astype(float)
reviews['date'] = pd.to_datetime(reviews['date'])
#calculate the average value of the rating column (A)
def mean_rating_A(reviews):
    sum_rating = 0.0
    count_rating = 0
    
    for index, row in reviews.iterrows():
        if row['date'].year == 2010:
            sum_rating += row['rating']
            count_rating += 1
    
    if count_rating == 0:
        return 0.0
    
    return sum_rating / count_rating

#Measuring the execution time of a function Option A
start_time = time.time()
result_A = mean_rating_A(reviews)
end_time = time.time()
print("Option A: Average rating for reviews left in 2010 - ", result_A)
print("Time to execute Option A: ", end_time - start_time, "seconds")
#B
def mean_rating_B(reviews_2010):
    sum_rating = 0.0
    count_rating = 0
    
    for index, row in reviews_2010.iterrows():
        sum_rating += row['rating']
        count_rating += 1
    
    if count_rating == 0:
        return 0.0
    
    return sum_rating / count_rating

start_time = time.time()
reviews_2010 = reviews[reviews['date'].dt.year == 2010]
result_B = mean_rating_B(reviews_2010)
end_time = time.time()
print("Option B: Average rating for reviews left in 2010 - ", result_B)
print("Time to execute Option B: ", end_time - start_time, "seconds")
#C
def mean_rating_C(reviews_2010):
    return reviews_2010['rating'].mean()

start_time = time.time()
reviews_2010 = reviews[reviews['date'].dt.year == 2010]
result_C = mean_rating_C(reviews_2010)
end_time = time.time()
print("Option C: Average rating for reviews left in 2010 - ", result_C)
print("Time to execute Option c: ", end_time - start_time, "seconds")

Option A: Average rating for reviews left in 2010 -  4.4544402182900615
Time to execute Option A:  9.230548620223999 seconds
Option B: Average rating for reviews left in 2010 -  4.4544402182900615
Time to execute Option B:  0.8662064075469971 seconds
Option C: Average rating for reviews left in 2010 -  4.4544402182900615
Time to execute Option c:  0.0193178653717041 seconds


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [4]:
import pandas as pd

df = pd.read_csv('reviews_sample.csv')


def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['user_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count
import time

start_time = time.time()
result = get_word_reviews_count(df)
end_time = time.time()

print(f"Execution time: {end_time - start_time} seconds")

Execution time: 27.093859434127808 seconds


4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.

In [4]:
import numpy as np

#Без использования векторизованных операций и методов массивов numpy и без использования numba:
def mape(ratings):
    total_sum = 0
    total_count = 0
    for rating in ratings:
        if rating != 0:
            total_sum += abs(rating - np.mean(ratings))
            total_count += 1
    return total_sum / total_count * 100 if total_count > 0 else 0


#С использованием numba:
import numba as nb

@nb.jit(nopython=True)
def mape_numba(ratings):
    total_sum = 0
    total_count = 0
    for rating in ratings:
        if rating != 0:
            total_sum += abs(rating - np.mean(ratings))
            total_count += 1
    return total_sum / total_count * 100 if total_count > 0 else 0


#С использованием векторизованных операций и методов массивов numpy, без использования numba:
def mape_np(ratings):
    ratings = ratings[ratings != 0]
    return np.mean(np.abs(ratings - np.mean(ratings))) * 100 if len(ratings) > 0 else 0


#С использованием векторизованных операций и методов массивов numpy и numba:
import numba as nb

@nb.jit(nopython=True)
def mape_np_numba(ratings):
    ratings = ratings[ratings != 0]
    return np.mean(np.abs(ratings - np.mean(ratings))) * 100 if len(ratings) > 0 else 0


#Для измерения времени выполнения каждой из функций можно использовать модуль timeit:
import timeit

ratings = np.random.randint(0, 5, 10000)

start_time = timeit.default_timer()
mape(ratings)
print("mape:", timeit.default_timer() - start_time)

start_time = timeit.default_timer()
mape_numba(ratings)
print("mape with numba:", timeit.default_timer() - start_time)

start_time = timeit.default_timer()
mape_np(ratings)
print("mape with numpy:", timeit.default_timer() - start_time)

start_time = timeit.default_timer()
mape_np_numba(ratings)
print("mape with numpy and numba:", timeit.default_timer() - start_time)

mape: 0.20501299400007156
mape with numba: 1.7949455139998918
mape with numpy: 0.0022335759999805305
mape with numpy and numba: 0.6340868380000302
